# Importuri

In [1]:
import torch
import numpy
import random
import treys
from treys import Deck
from treys import Card
from treys import Evaluator
import matplotlib
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from tensorflow import keras
import numpy as np

#  Implementarea Agentului

In [25]:
epsilons = []

In [136]:
class DnqAgent:
    def __init__(self):
        self.evaluator_agent = Evaluator()
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.99
        self.epsilon_min = 0.001
        self.epsilon_max = 1
        self.learning_rate = 0.4
        self.action_model = self._bulild_model()
        self.last_action = None
        self.hand_power =None
        self.batch_size = 9
        self.discount = 0.25
        self.own_memory = deque(maxlen=1024)#memoria agentului
        self.games_played = 0
    
    def update_epsilon(self):
        self.games_played += 1
        self.epsilon = max(self.epsilon_max - (self.games_played * (self.epsilon_max / self.epsilon_decay)),self.epsilon_min)
    
    def play(self,state):
        if random.random() < self.epsilon:
            action = random.choice(range(3))
        else:
            action = np.argmax(self.action_model.predict(np.asarray([state]))[0])
        self.update_epsilon()
        epsilons.append(self.epsilon)
        return action
    
    
    def _bulild_model(self):
        model = keras.Sequential()
        model.add(keras.layers.InputLayer(input_shape=(9,)))#None, 9)))#(9,)))
        model.add(keras.layers.Dense(4,activation = "sigmoid"))
        model.add(keras.layers.Dense(3,activation = "sigmoid"))
        model.compile(loss = "mse",optimizer = keras.optimizers.Adam(learning_rate=self.gamma))
        model.summary()
        return model
    
                                   
    def remember(self,state,action,reward,following_state, done):
        self.own_memory.append([state,action,reward,following_state, done])
                                   
    def replay(self):
        auxiliary_batch = random.sample(self.own_memory,self.batch_size)
        for state,action,reward,following_state,done in auxiliary_batch:
            target = reward
            if not done:
                target = (reward+self.gamma * np.amax(self.action_model.predict(following_state)[0]))
            target_f = self.action_model.predict(state)
            target_f[0][action] = target
            
            self.action_model.fit(state,target_f,epochs =1,verbose = 0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *=self.epsilon_decay
            epsilons.append(self.epsilon)
    
    
                                   
    def _peek(self,hand,table):
        self.hand_power = 1-(self.evaluator_agent.evaluate(hand,table)-1)/7462
    
                                   
    def make_state(self,previous_oponent_action,table,pot,stage):
        if stage == "flop":
            state = np.concatenate(([1,0,0],[self.hand_power],[previous_oponent_action],[self.last_action],[pot],[AceOnTable(table)],[KingOnTable(table)]))
        elif stage == "turn":
            state = np.concatenate(([0,1,0],[self.hand_power],[previous_oponent_action],[self.last_action],[pot],[AceOnTable(table)],[KingOnTable(table)]))
        elif stage == "river":
            state = np.concatenate(([0,0,1],[self.hand_power],[previous_oponent_action],[self.last_action],[pot],[AceOnTable(table)],[KingOnTable(table)]))
        else:
             state = np.concatenate(([0,0,0],[self.hand_power],[previous_oponent_action],[self.last_action],[pot],[AceOnTable(table)],[KingOnTable(table)]))
        state =tuple(state)
        state = np.reshape(state,[1,9]).astype(np.float32)
        return state
    
                                   

### Cateva functii si clase ajutatoare in simularea environment-ului

In [47]:
class PassInfo:
    def _init_(self):
        self.round = None
        self.precedentAction= None
        self.potTotal = None
        self.activePlayer = None
        self.lastBet = None
        
    def NextPlayer(self):
        self.activePlayer = (self.activePlayer +1 )%2
    
    def AddToPot(self,bet):
        self.potTotal = self.potTotal + bet
    
    def CheckToRaise(self):
        self.potTotal = self.potTotal + self.lastBet
        
    def Raise(self,bet):
        self.AddToPot(self.lastBet)
        self.AddToPot(bet)
        self.lastBet = bet

In [48]:
def AceOnTable(table):
    for ace in [Card.new('Ah'),Card.new('As'),Card.new('Ad'),Card.new('Ac')]:
        if ace in table:
            return 1
    return 0

def KingOnTable(table):
    for king in [Card.new('Kh'),Card.new('Ks'),Card.new('Kd'),Card.new('Kc')]:
        if king in table:
            return 1
    return 0

In [49]:
def TableAction(ActionTaken,InfoToPass):
    #print("AmAjunslaTableAction")
    if ActionTaken == 0:
        return False
    elif ActionTaken == 1:
        if InfoToPass.precedentAction == 1:
            return False
        else:
            if InfoToPass.precedentAction == 2:
                InfoToPass.CheckToRaise()
                InfoToPass.NextPlayer()
                return False
    elif ActionTaken == 2:
        InfoToPass.NextPlayer()
        return True

In [50]:
def BetRoundAgent(Agent,InfoToPass,board):
    #print("AmAjunslaPariulAgentului")
    current_state = Agent.make_state(InfoToPass.precedentAction,board,InfoToPass.potTotal,InfoToPass.round)
    action = Agent.play(current_state)
    Agent.last_action =action
    InfoToPass.precedentAction = action
    if action == 2:
        bet = random.randint(1,10)
        InfoToPass.Raise(bet)
    return current_state,action
     

In [51]:
def BetRoundOponent(DummyAgent,InfoToPass,table):
    #print("AmAjunsLaPariulTrainerului")
    action = DummyAgent.action(table)
    InfoToPass.precedentAction = action
    if action == 2:
        bet = random.randint(1,10)
        InfoToPass.Raise(bet)
    return action

In [52]:
blinds = [1,0]

In [58]:
def SimulateGame(Agent,DummyAgent):
    print("NewGameBegan")
    InfoToPass = PassInfo()
    InfoToPass.lastBet = 1
    InfoToPass.precedentAction = 2
    deck = Deck()
    deck.shuffle()
    trainer_hand=deck.draw(2)
    agent_hand=deck.draw(2)
    DummyAgent.look_at_hand(trainer_hand)
    InfoToPass.potTotal = sum(blinds)
    if blinds[0] == 0:
        InfoToPass.activePlayer = 0
    else:
        InfoToPass.activePlayer = 1
    potAgent = 0
    #Agent._peek(agent_hand,table)
    
        
    #the flop
    InfoToPass.round = "flop"
    deck.draw(1)
    table=deck.draw(3)
    Agent._peek(agent_hand,table)
    
    betting_round_going = True
    if InfoToPass.activePlayer == 0:
        while betting_round_going:
            action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            next_state,action = BetRoundAgent(Agent,InfoToPass,table)
            BetRoundAgent(Agent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going:
                done = False
            else:
                if action == 0:
                    done = True
                    reward = -1*InfoToPass.potTotal
                else:
                    done = True
                    reward = 0
            reward = reward if done else 0
            rewards.append(reward)
            Agent.remember(state,action,reward,next_state, done)
            state = next_state
            if Agent.games_played > 2*Agent.batch_size:
                Agent.replay()
    else:
        while betting_round_going:
            next_state,action = BetRoundAgent(Agent,InfoToPass,table)
            BetRoundAgent(Agent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going:
                done = False
            else:
                if action == 0:
                    done = True
                    reward = -1*InfoToPass.potTotal
                else:
                    done = True
                    reward = 0
            action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            reward = reward if done else 0
            rewards.append(reward)
            state = Agent.make_state(InfoToPass.precedentAction,table,InfoToPass.potTotal,InfoToPass.round)
            Agent.remember(state,Agent.last_action,reward,next_state, done)
            state = next_state
            if Agent.games_played > 2*Agent.batch_size:
                Agent.replay()
    #the turn
    InfoToPass.round="turn"
    deck.draw(1)
    table.append(deck.draw(1))
    Agent._peek(agent_hand,table)
    
    betting_round_going = True
    if InfoToPass.activePlayer == 0:
        while betting_round_going:
            action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            next_state,action = BetRoundAgent(Agent,InfoToPass,table)
            BetRoundAgent(Agent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going:
                done = False
            else:
                if action == 0:
                    done = True
                    reward = -1*InfoToPass.potTotal
                else:
                    done = True
                    reward = 0
            reward = reward if done else 0
            rewards.append(reward)
            Agent.remember(state,action,reward,next_state, done)
            state = next_state
            if Agent.games_played > 2*Agent.batch_size:
                Agent.replay()
    
    else:
        while betting_round_going:
            next_state,action = BetRoundAgent(Agent,InfoToPass,table)
            BetRoundAgent(Agent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going:
                done = False
            else:
                if action == 0:
                    done = True
                    reward = -1*InfoToPass.potTotal
                else:
                    done = True
                    reward = 0
            if betting_round_going == False:
                action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            reward = reward if done else 0
            rewards.append(reward)
            Agent.remember(state,Agent.last_action,reward,next_state, done)
            state = next_state
            if Agent.games_played > 2*Agent.batch_size:
                Agent.replay()
        
    #the river
    InfoToPass.round = "river"
    deck.draw(1)
    table.append(deck.draw(1))
    Agent._peek(agent_hand,table)
    betting_round_going = True
    if InfoToPass.activePlayer == 0:
        while betting_round_going:
            action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            if betting_round_going:
                next_state,action = BetRoundAgent(Agent,InfoToPass,table)
                BetRoundAgent(Agent,InfoToPass,table)
                betting_round_going=TableAction(action,InfoToPass)
                if betting_round_going:
                    done = False
                else:
                    if action == 0:
                        done = True
                        reward = -1*InfoToPass.potTotal
                    else:
                        done = True
                        reward = 0
                reward = reward if done else 0
                rewards.append(reward)
                Agent.remember(state,action,reward,next_state, done)
                state = next_state
                if Agent.games_played > A2*gent.batch_size:
                    Agent.replay()
    else:
        while betting_round_going:
            next_state,action = BetRoundAgent(Agent,InfoToPass,table)
            BetRoundAgent(Agent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going:
                done = False
            else:
                if action == 0:
                    done = True
                    reward = -1*InfoToPass.potTotal
                else:
                    done = True
                    evaluator = Evaluator()
                    if evaluator.evaluate(table,trainer_hand) < evaluator.evaluate(table,agent_hand):
                        reward = InfoToPass.potTotal
                    else:
                        reward = -1*InfoToPass.potTotal
            if betting_round_going:
                action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            reward = reward if done else 0
            rewards.append(reward)
            Agent.remember(state,Agent.last_action,reward,next_state, done)
            state = next_state
            if Agent.games_played > 2*Agent.batch_size:
                Agent.replay()
                

#### Simulari a diferite abordari a jocului,cu scop de antrenament

In [76]:
class SafePlayerSimulator:
    def __init__(self):
        self.hand = None
        self.evaluator = Evaluator()
        
    def look_at_hand(self,cards):
        self.hand= cards
    
    def action(self,table):
        if 1-(self.evaluator.evaluate(table, self.hand)-1)/7462 >  0.75:
            return 2
        elif 1-(self.evaluator.evaluate(table, self.hand)-1)/7462 >  0.5:
            return 1
        else:
            return 0
        

class AgressivePlayerSimulator:
    def __init__(self):
        self.hand = None
        self.evaluator = Evaluator()
    
    def look_at_hand(self,cards):
        self.hand= cards
    
    def action(self,table):
        if 1-(self.evaluator.evaluate(table, self.hand)-1)/7462 >  0.5:
            return 1
        elif 1-(self.evaluator.evaluate(table, self.hand)-1)/7462 >  0.25:
            return 2
        else:
            return 0

class PassivePlayerSimulator:
    def __init__(self):
        self.hand = None
        self.evaluator = Evaluator()
        
    def look_at_hand(self,cards):
        self.hand= cards
    
    def action(self,table):
        if 1-(self.evaluator.evaluate(table, self.hand)-1)/7462 >  0.9:
            return 2
        elif 1-(self.evaluator.evaluate(table, self.hand)-1)/7462 >  0.7:
            return 1
        else:
            return 0

# Antrenarea Agentului

In [140]:
rewards = []
trainer1 = SafePlayerSimulator()
trainer2 = AgressivePlayerSimulator()
trainer3 = PassivePlayerSimulator()
Agent = DnqAgent()
for attempt in range(300):
    print(Agent.games_played)
    if attempt % 3 == 1:
        SimulateGame(Agent,trainer1)
    elif attempt%3==2:
        SimulateGame(Agent,trainer2)
    else:
        SimulateGame(Agent,trainer3)
   
        
plt.title("Reward evolution")
plt.xlabel("total reward")
plt.ylabel("decisions made")
plt.plot(np.cumsum(rewards),[i+1 for i in range(len(rewards))], color ="green")
plt.show()


plt.title("epsilon evolution")
plt.xlabel("epsilon")
plt.ylabel("decisions made")
plt.plot(epsilons,[i+1 for i in range(len(epsilons))], color ="green")
plt.show()

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_98 (Dense)             (None, 4)                 40        
_________________________________________________________________
dense_99 (Dense)             (None, 3)                 15        
Total params: 55
Trainable params: 55
Non-trainable params: 0
_________________________________________________________________
0
NewGameBegan
4
NewGameBegan
10
NewGameBegan
16
NewGameBegan


Exception ignored in: <function ScopedTFGraph.__del__ at 0x000002139B415F70>
Traceback (most recent call last):
  File "D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
AttributeError: deleter


22
NewGameBegan
28
NewGameBegan


KeyboardInterrupt: 

# Experimentati agentul

# Spare code

In [90]:
#blind betting
    betting_round_going = True
    if InfoToPass.activePlayer == 0:
        while betting_round_going:
            action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            next_state,action = BetRoundAgent(Agent,InfoToPass)
            BetRoundAgent(Agent,InfoToPass)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going:
                done = False
            else:
                if action == 0:
                    done = True
                    reward = -1*InfoToPass.potTotal
                else:
                    done = True
                    reward = 0
            reward = reward if done else 0
            rewards.append(reward)
            Agent.remember(state,action,reward,next_state, done)
            state = next_state
            if Agent.games_played > Agent.batch_size:
                Agent.replay()
    else:
        while betting_round_going:
            next_state,action = BetRoundAgent(Agent,InfoToPass)
            BetRoundAgent(Agent,InfoToPass)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going:
                done = False
            else:
                if action == 0:
                    done = True
                    reward = -1*InfoToPass.potTotal
                else:
                    done = True
                    reward = 0
            action = BetRoundOponent(DummyAgent,InfoToPass,table)
            betting_round_going=TableAction(action,InfoToPass)
            if betting_round_going == False:
                done = True
                reward = InfoToPass.potTotal
            reward = reward if done else 0
            rewards.append(reward)
            Agent.remember(state,Agent.last_action,reward,next_state, done)
            state = next_state
            if Agent.games_played > Agent.batch_size:
                Agent.replay()

IndentationError: unexpected indent (Temp/ipykernel_3956/2661479192.py, line 2)